In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# **1. 데이터 로드 및 전처리**
***
> 데이터를 로드하고 자이로스코프 데이터를 선택하고, 이를 정규화

In [10]:
# 데이터 읽기 및 전처리
def load_and_preprocess_data(file_path):
    """
    CSV 파일에서 데이터를 로드하고 자이로스코프 데이터를 선택하여 정규화한 후, 
    추가적인 특징을 생성하는 함수
    """

    # 숫자형 데이터만 로드하기 위해 dtype을 지정
    data = pd.read_csv(file_path, dtype={
        'gyros_forearm_x': float,
        'gyros_forearm_y': float,
        'gyros_forearm_z': float
    }, low_memory=False)

    # 사용할 특징: 자이로스코프 데이터
    features = data[['gyros_forearm_x', 'gyros_forearm_y', 'gyros_forearm_z']]
    labels = data['classe']

    # 데이터가 숫자형으로 변환되지 않은 비정상적인 값을 포함하고 있는지 확인
    print(features.dtypes)
    print(features.isnull().sum())

    # 데이터 정규화: 각 특징을 평균이 0, 표준편차가 1이 되도록 변환
    features = (features - features.mean()) / features.std()

    return features, labels

# pml-training 데이터 로드 및 전처리
training_data_path = './원본 데이터/pml-training.csv'
features, labels = load_and_preprocess_data(training_data_path)

gyros_forearm_x    float64
gyros_forearm_y    float64
gyros_forearm_z    float64
dtype: object
gyros_forearm_x    0
gyros_forearm_y    0
gyros_forearm_z    0
dtype: int64


# **2. 특징 공학 (Feature Engineering)**
***
> 추가적인 특징을 생성하여 모델에 더 유의미한 입력을 제공
> 각 축에 대한 통계적 특징(평균, 표준편차, 최댓값, 최솟값, 범위)과 자이로스코프 데이터의 벡터 크기를 계산하여 추가

In [13]:
# 추가적인 특징 생성 함수
def feature_engineering(df):
    """
    자이로스코프 데이터에서 추가적인 특징을 생성하는 함수
    """

    # 각 축에 대해 통계량 계산
    for axis in ['x', 'y', 'z']:
        df[f'gyros_forearm_{axis}_mean'] = df[f'gyros_forearm_{axis}'].mean()
        df[f'gyros_forearm_{axis}_std'] = df[f'gyros_forearm_{axis}'].std()
        df[f'gyros_forearm_{axis}_max'] = df[f'gyros_forearm_{axis}'].max()
        df[f'gyros_forearm_{axis}_min'] = df[f'gyros_forearm_{axis}'].min()
        df[f'gyros_forearm_{axis}_range'] = df[f'gyros_forearm_{axis}_max'] - df[f'gyros_forearm_{axis}_min']

    # 벡터 크기 계산
    df['gyros_forearm_magnitude'] = np.sqrt(df['gyros_forearm_x'] ** 2 + df['gyros_forearm_y'] ** 2 + df['gyros_forearm_z'] ** 2)

    return df

# 추가적인 특징 생성
features = feature_engineering(features)

# **3. 데이터 분할 및 라벨 인코딩**
***
> 데이터를 학습용과 테스트용으로 분할하고, 라벨을 신경망에서 사용할 수 있도록 원-핫 인코딩을 수행

In [14]:
# 데이터 분할 및 라벨 인코딩
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

# 입력 데이터 차원 확장
x_train = np.expand_dims(x_train, axis=1) # (샘플 수, 시퀸스 길이, 1)로 변환
x_test = np.expand_dims(x_test, axis=1) # (샘플 수, 시퀸스 길이, 1)로 변환

# **4. 모델 생성 및 학습**
***
> 이 단계에서는 CNN+LSTM 구조의 모델을 생성하고, 학습을 진행, 모델의 성능을 최적화하기 위해 조기 종료(Early Stopping)와 모델 체크포인트 저장(Model Checkpoint)을 사용

In [16]:
# CNN+LSTM 모델 생성 함수
def create_cnn_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Conv1D(filters=64, kernel_size=2, strides=1, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool1D(pool_size=1),
        
        tf.keras.layers.Conv1D(filters=64, kernel_size=2, strides=1, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool1D(pool_size=1),

        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dropout(0.5),
        
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dropout(rate=0.5),
        tf.keras.layers.Dense(units=num_classes, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 모델 생성
input_shape = (x_train.shape[1], x_train.shape[2])
num_classes = y_train.shape[1]
model = create_cnn_model(input_shape, num_classes)
model.summary()

# 콜백 함수 정의
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

# 모델 학습
history = model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, model_checkpoint],
    verbose=2
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 1, 64)             2496      
                                                                 
 batch_normalization_2 (Batc  (None, 1, 64)            256       
 hNormalization)                                                 
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 1, 64)            0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 1, 64)             8256      
                                                                 
 batch_normalization_3 (Batc  (None, 1, 64)            256       
 hNormalization)                                                 
                                                      

# **5. 모델 평가**
***
> 학습된 모델을 평가하여 훈련데이터와 테스트 데이터에서의 성능을 측정

In [17]:
# 훈련 데이터와 테스트 데이터에 대한 성능 평가
train_loss, train_acc = model.evaluate(x_train, y_train, verbose=2)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print(f"훈련 정확도: {train_acc}, 테스트 정확도: {test_acc}")

491/491 - 1s - loss: 1.5374 - accuracy: 0.3220 - 824ms/epoch - 2ms/step
123/123 - 0s - loss: 1.5439 - accuracy: 0.3167 - 212ms/epoch - 2ms/step
훈련 정확도: 0.3219723403453827, 테스트 정확도: 0.3166879117488861
